
# Employer Matching

Unlike stable marriage problem, this does not require the preferences to be unique (aka one person can have same score of preference for the other).

https://github.com/alextanhongpin/stable-marriage-problem


## References
https://towardsdatascience.com/how-to-match-two-people-with-python-7583b51ff3f9

In [72]:
from collections import defaultdict

import numpy as np
import pandas as pd

from ortools.sat.python import cp_model

In [73]:
# The diagonal is always zero since you cannot rate yourself.
names = ["Ben", "Kate", "Thinh", "Jorge", "Alfredo", "Francisco", "Olivia", "Chris"]
data = np.array(
    [
        [0, 0, 3, 3, 7, 9, 3, 5],
        [2, 0, 7, 6, 8, 8, 1, 6],
        [7, 7, 0, 1, 5, 9, 8, 9],
        [4, 3, 0, 0, 5, 0, 2, 3],
        [8, 1, 3, 3, 0, 7, 0, 1],
        [9, 9, 0, 4, 7, 0, 2, 7],
        [2, 0, 0, 4, 5, 5, 0, 8],
        [4, 1, 4, 9, 8, 1, 1, 0],
    ]
)

df = pd.DataFrame(data, columns=names, index=names)
df

,Ben,Kate,Thinh,Jorge,Alfredo,Francisco,Olivia,Chris
Ben,0,0,3,3,7,9,3,5
Kate,2,0,7,6,8,8,1,6
Thinh,7,7,0,1,5,9,8,9
Jorge,4,3,0,0,5,0,2,3
Alfredo,8,1,3,3,0,7,0,1
Francisco,9,9,0,4,7,0,2,7
Olivia,2,0,0,4,5,5,0,8
Chris,4,1,4,9,8,1,1,0


In [119]:
model = cp_model.CpModel()

people = range(8)
matches = defaultdict(dict)

for i in people:
    for j in people:
        matches[i][j] = model.NewBoolVar(f"{i} with {j}")

# One person can only be matched with another.
for i in people:
    model.Add(sum(matches[i][j] for j in people) == 1)
    model.Add(sum(matches[j][i] for j in people) == 1)

    # Cannot match yourself.
    model.Add(matches[i][i] == 0)

# If a match with b, b must match with a.
for i in people:
    for j in people:
        model.Add(matches[i][j] == matches[j][i])

model.Maximize(sum([matches[i][j] * data[i, j] for i in people for j in people]))

In [120]:
class MatchingSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, people, names, matches):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._solution_count = 0
        self._people = people
        self._matches = matches
        self._names = names

    def on_solution_callback(self):
        self._solution_count += 1
        print("Solution %i" % self._solution_count)
        cached = {}
        for i in self._people[:4]:
            for j in self._people:
                if self.Value(self._matches[i][j]):
                    print(self._names[i], "matches", self._names[j])

In [121]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
solution_printer = MatchingSolutionPrinter(people, names, matches)

status = solver.Solve(model, solution_printer)
if status == cp_model.INFEASIBLE:
    print("Solution is infeasible")
elif status == cp_model.FEASIBLE:
    print("Solution is feasible")
elif status == cp_model.OPTIMAL:
    print("Solution is optimal")

Solution 1
Ben matches Alfredo
Kate matches Francisco
Thinh matches Olivia
Jorge matches Chris
Solution is optimal


In [122]:
cached = {}
for i in people[:4]:
    for j in people:
        if solver.Value(matches[i][j]):
            print(
                names[i],
                "matches",
                names[j],
                "with scores",
                data[i][j],
                "and",
                data[j][i],
                "totalling",
                data[i][j] + data[j][i],
            )

Ben matches Alfredo with scores 7 and 8 totalling 15
Kate matches Francisco with scores 8 and 9 totalling 17
Thinh matches Olivia with scores 8 and 0 totalling 8
Jorge matches Chris with scores 3 and 9 totalling 12
